In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import pandas as pd
import geopandas as gpd

In [3]:
import json

In [4]:
from datetime import datetime, date
import os
import numpy as np
import imageio

In [5]:
import matplotlib.colors as colors
from datetime import timedelta, date, datetime
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable

In [6]:
def loadGeopandas():    
    shapefile = 'data/countries_110m/ne_110m_admin_0_countries.shp'
    gdf = gpd.read_file(shapefile)[['ADMIN', 'ADM0_A3', 'geometry']]
    gdf.head()
    gdf.columns = ['country', 'country_code', 'geometry']
    gdf.head()
    
    print(gdf[gdf['country'] == 'Antarctica'])
    #Drop row corresponding to 'Antarctica'
    gdf = gdf.drop(gdf.index[159])
    return gdf

# Get data from CSSEGISandData

In [7]:
normalize_population = False # swicth it on when want to normalise by population

In [8]:
def getTimeSeries(fname):
    print(fname)
    df = pd.read_csv(fname)
    countryData = df.groupby('Country/Region').sum()
    countryData = countryData.drop([ 'Lat', 'Long'], axis=1)
    return countryData.T
    
def fetchCountryData(freq='1d'):
    gisandata ='https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_'
    confirmed = getTimeSeries(f'{gisandata}confirmed_global.csv')
    return confirmed

In [9]:
def get_strdate(val):
    mounth = val.strftime("%m")
    day = val.strftime("%d")
    yer = val.strftime("%y")
    
    if mounth[0] == '0':
        mounth = mounth[1]
    if day[0] == '0':
        day = day[1]
    return '/'.join([mounth,day, yer])

In [10]:
from countryinfo import CountryInfo
import pycountry

In [11]:
# Load all datas
gdf = loadGeopandas()

data = fetchCountryData()
data = data.rename(columns={'Country/Region': 'country_region'})
countries_code = pd.read_csv('JohnsHopkins-to-A3.csv')

        country country_code  \
159  Antarctica          ATA   

                                              geometry  
159  MULTIPOLYGON (((-48.66062 -78.04702, -48.15140...  
https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv


In [12]:
pop = []
missing_pop = []

df = pd.read_csv('population.csv')
for i, row in countries_code.iterrows():
    a3 = row['alpha3']
    p = df[(df['Code']==a3) &( df['Year']=='2019')]['Population']
    try:
        pop.append(p.values[0])
    except IndexError:
        pop.append(1)
        missing_pop.append(a3)
# print(pop)
countries_code['population'] = pop


In [13]:

def mergegdf_bydate(dateval, norm=False):
    # dateval as str

    max_conf_bydate = 0
    min_conf_bydate = 10
    # tmp_df = data[[r'Country/Region', dateval]].copy()
    tmp_df = data.loc[dateval]
    # tmp_df = tmp_df.rename(columns={dateval: 'confirmed'})
    tmp_df_code = pd.merge(countries_code, tmp_df, on='Country/Region')

    tmp_df_code[dateval] = pd.to_numeric(tmp_df_code[dateval], downcast="float")
    if norm:  # if to normalise to population
        for i, row in tmp_df_code.iterrows():
            val_percap = float(row[dateval]) / float(row['population'])
            if row['population'] > 1 and row[dateval] >0:
                if val_percap > max_conf_bydate:
                    max_conf_bydate = val_percap
                if val_percap < min_conf_bydate:
                    min_conf_bydate = val_percap
            if row['population'] == 1:
                val_percap = 0.
            tmp_df_code.at[i, dateval] = val_percap
    else:
        max_conf_bydate = max(tmp_df_code[dateval])
    merged = gdf.merge(tmp_df_code, left_on='country_code', right_on='alpha3', how='left')
    merged = merged.rename({'Country/Region': 'country'})

    #     merged.fillna(0, inplace = True)
    return merged, (min_conf_bydate, max_conf_bydate)

In [14]:
def get_max():
    yesterday  = datetime.date(datetime.now()-timedelta(1))
    _, maxconf = mergegdf_bydate(get_strdate(yesterday))
    return maxconf[1]


In [15]:
def create_gif(odir, start, end, images):
    exportname = '{}/map_{}_{}_2.gif'.format(odir, start, end)
    kargs = { 'duration': 0.7 }
    imageio.mimsave(exportname, images[::2], 'GIF', **kargs)

    print('Checkout your new gif', exportname)


In [16]:

def create_imgs(single_date, fname, range, norm=False):
    date = get_strdate(single_date)
    merged,_ = mergegdf_bydate(date, norm)

    fig, ax = plt.subplots(1, 1, figsize=(20, 12))

    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right", size="2.5%", pad=0.01)

    title = 'COVID-19 cases on {}'.format(single_date)
    cz_label = 'Number of confirmed cases'
    cm = 'inferno_r'
    if norm_by_pop:
        title = 'COVID-19 cases per capita on {}'.format(single_date)
        cz_label = 'N. of confirmed cases per capita'
        cm = 'viridis_r'
    merged.plot(column=date, cmap=cm,
                linewidth=0.6, ax=ax, edgecolor='0.8',
                legend=True,
                norm=colors.LogNorm(vmin=range[0], vmax=range[1]),
                cax=cax)

    ax.axis('off')

    ax.text(0.3, 1.1, title,
            fontdict={'fontsize': 25, 'fontweight': '3'},
            transform=ax.transAxes)

    ax.annotate('Source: CSSEGISandData', xy=(0.1, .1),
                xycoords='figure fraction', horizontalalignment='left',
                verticalalignment='top', fontsize=12, color='#555555')

    ax.annotate('Copyright: RizM', xy=(0.1, .05),
                xycoords='figure fraction', horizontalalignment='left',
                verticalalignment='top', fontsize=12, color='#555555')

    cax.tick_params(labelsize=18)
    cax.set_ylabel(cz_label, fontsize=18)
    #     cax.labelsize=18)
    odir = fname.split('/')[0]
    os.makedirs(odir, exist_ok=True)
    fig.savefig(fname)
    plt.close(fig)


Get csv with the countries and alpha codes

In [17]:


def minMax(drange):

    min_percapita = 10
    max_percapita = 0
    for d in drange:
        dt = get_strdate(d)
        _, range = mergegdf_bydate(dt, True)
        if  range[0] < min_percapita and range[0]>0:
            min_percapita = range[0]

        if range[1] > max_percapita:
            max_percapita = range[1]
    return (min_percapita, max_percapita)



In [18]:

def daterange(start_date, end_date):
    drange = []
    for n in range(int((end_date - start_date).days)):
        drange.append(start_date + timedelta(n))
    return drange



today = datetime.date(datetime.now())
start = date(2020, 1, 22)
end = date(2020, 1, 30)

drange = daterange(start,today)
images = []
# '''
norm_by_pop = False

if norm_by_pop:
    range = minMax(drange)
else:
    range=(1,  get_max())

for single_date in drange:
    odir = 'covid_map'
    if norm_by_pop:
        odir = 'covid_map_percapita'
    fname = f'{odir}/{single_date.strftime("%d_%m_%Y")}.png'

    print(single_date)
    create_imgs(single_date, fname, range, norm_by_pop)

    images.append(imageio.imread(fname))

create_gif(odir, start, today, images)


2020-01-22
2020-01-23
2020-01-24
2020-01-25
2020-01-26
2020-01-27
2020-01-28
2020-01-29
2020-01-30
2020-01-31
2020-02-01
2020-02-02
2020-02-03
2020-02-04
2020-02-05
2020-02-06
2020-02-07
2020-02-08
2020-02-09
2020-02-10
2020-02-11
2020-02-12
2020-02-13
2020-02-14
2020-02-15
2020-02-16
2020-02-17
2020-02-18
2020-02-19
2020-02-20
2020-02-21
2020-02-22
2020-02-23
2020-02-24
2020-02-25
2020-02-26
2020-02-27
2020-02-28
2020-02-29
2020-03-01
2020-03-02
2020-03-03
2020-03-04
2020-03-05
2020-03-06
2020-03-07
2020-03-08
2020-03-09
2020-03-10
2020-03-11
2020-03-12
2020-03-13
2020-03-14
2020-03-15
2020-03-16
2020-03-17
2020-03-18
2020-03-19
2020-03-20
2020-03-21
2020-03-22
2020-03-23
2020-03-24
2020-03-25
2020-03-26
2020-03-27
2020-03-28
2020-03-29
2020-03-30
2020-03-31
2020-04-01
2020-04-02
2020-04-03
2020-04-04
2020-04-05
2020-04-06
2020-04-07
2020-04-08
2020-04-09
2020-04-10
2020-04-11
2020-04-12
2020-04-13
2020-04-14
2020-04-15
2020-04-16
2020-04-17
2020-04-18
2020-04-19
2020-04-20
2020-04-21



<br>
import qgrid<br>
qgrid_w = qgrid.show_grid(tmp_df, show_toolbar = True)<br>
qgrid_w<br>
